In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
# ------------1. GroupBy 技术---------------

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.493858,1.071114,a,one
1,0.213009,-0.113068,a,two
2,1.288731,-0.764188,b,one
3,0.702308,1.687314,b,two
4,1.322865,-0.106289,a,one


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped  # 返回一个GroupBy对象，还没有进行计算，只是一个分组中间数据

In [5]:
# 调用mean方法进行分组平均值
grouped.mean()

key1
a    1.009910
b    0.995519
Name: data1, dtype: float64

In [6]:
#  一次传入多个数组
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.408361
      two     0.213009
b     one     1.288731
      two     0.702308
Name: data1, dtype: float64

In [7]:
# 或者传入列名数组
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.408361  0.482413
     two   0.213009 -0.113068
b    one   1.288731 -0.764188
     two   0.702308  1.687314

In [8]:
# 返回分组大小的Series(含有层次化的索引)
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [12]:
# 转化一下,就变成计数了
size = df.groupby(['key1', 'key2']).size()
size.unstack()

key2,one,two
key1,,
a,2,1
b,1,1


In [13]:
#  -----------------2.对分组进行迭代 Iterating Over Groups----------------

In [16]:
# 可以迭代，产生一组 二元元组（分组名 + 数据块）
for name, group in df.groupby('key1'):
    print(name) # key1的值
    print(group) # 分组结果的数据块

a
      data1     data2 key1 key2
0  1.493858  1.071114    a  one
1  0.213009 -0.113068    a  two
4  1.322865 -0.106289    a  one
b
      data1     data2 key1 key2
2  1.288731 -0.764188    b  one
3  0.702308  1.687314    b  two


In [18]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1,k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  1.493858  1.071114    a  one
4  1.322865 -0.106289    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.213009 -0.113068    a  two
('b', 'one')
      data1     data2 key1 key2
2  1.288731 -0.764188    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.702308  1.687314    b  two


In [19]:
# 将GroupBy对象转化为字典
pieces = dict(list(df.groupby('key1')))

pieces

{'a':       data1     data2 key1 key2
 0  1.493858  1.071114    a  one
 1  0.213009 -0.113068    a  two
 4  1.322865 -0.106289    a  one, 'b':       data1     data2 key1 key2
 2  1.288731 -0.764188    b  one
 3  0.702308  1.687314    b  two}

In [20]:
pieces['a']

,data1,data2,key1,key2
0,1.493858,1.071114,a,one
1,0.213009,-0.113068,a,two
4,1.322865,-0.106289,a,one


In [21]:
# ------------------3. 选取一个或者一组列------------

In [23]:
df.groupby('key1')['data1'].mean() 

key1
a    1.009910
b    0.995519
Name: data1, dtype: float64

In [25]:
# 等同于
df['data1'].groupby(df['key1']).mean()

key1
a    1.009910
b    0.995519
Name: data1, dtype: float64

In [26]:
# ------------- 4. 通过字典或者Series进行分组-------------------

In [29]:
people = pd.DataFrame(np.arange(25).reshape(5,5),columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0,1,2,3,4
Steve,5,6,7,8,9
Wes,10,11,12,13,14
Jim,15,16,17,18,19
Travis,20,21,22,23,24


In [30]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)  # 将字典传入
by_column.sum()

,blue,red
Joe,5,5
Steve,15,20
Wes,25,35
Jim,35,50
Travis,45,65


In [31]:
# 同样可以传入Series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [33]:
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,5,5
Steve,15,20
Wes,25,35
Jim,35,50
Travis,45,65


In [34]:
# -------------  5. 通过函数进行分组--------------------

In [35]:
people.groupby(len).sum() # 根据人名字的长度进行分组，只需要传入len()函数'

,a,b,c,d,e
3,25,28,31,34,37
5,5,6,7,8,9
6,20,21,22,23,24


In [36]:
# --------------6. 根据索引级别分组-----------------

In [37]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.609288 -0.190963  1.021093 -0.153247 -0.116746
1     -0.191140 -0.890589  0.844799  0.334571  0.324093
2      1.820125 -0.332608 -0.106604  0.733670  0.131415
3     -0.184120  3.187649 -0.356466  1.292187  0.713840

In [38]:
hier_df.groupby(level='cty', axis=1).count() # 通过level关键字传入级别编号或者名称

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [39]:
# ---------------------7.数据聚合------------------

In [40]:
df

,data1,data2,key1,key2
0,1.493858,1.071114,a,one
1,0.213009,-0.113068,a,two
2,1.288731,-0.764188,b,one
3,0.702308,1.687314,b,two
4,1.322865,-0.106289,a,one


In [42]:
# 除了调用groupby的方法，如：mean,sum,count,median,std.....
# 还可以传入自己的聚合函数
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped = df.groupby('key1')
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.280849,1.184182
b,0.586423,2.451501


In [43]:
# ------------8 . 分组transform 和 apply --------------------

In [44]:
# Apply: General split-apply-combine


In [45]:
# --------------9. 透视表和交叉表---------------

In [46]:
# 交叉表：主要用于计数统计
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [47]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [48]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10
